In [1]:
import openpyxl
import glob
import os
from openpyxl import Workbook
import numpy as np
import xlwings as xw
from openpyxl.styles import PatternFill

In [2]:
#リテンションタイムで整列させるファイルのパス名を入力してください
file="/Users/Miyo/Desktop/ガスクロ　プログラム/3.まとめ/比較用.xlsx"

#保存先のパス名
savepath="/Users/Miyo/Desktop/ガスクロ　プログラム/4.比較/比較.xlsx"
originalfilepath="/Users/Miyo/Desktop/ガスクロ　プログラム/0.textから変換済み"

#色付けしたいCAS番号を入力してください
given_CAS = ['928 - 96 - 1','3681 - 82 - 1','57 - 06 - 7',
            '61315 - 63 - 7']

In [3]:
"""
３.まとめたファイルを整列させます。
"""

def getRt_CAS_cols(cols_list):
    for i_list in [Rt_time_list,normal_list,small_list,big_list,Rt_time_value,CAS_val,CAS_list]:
        i_list.clear()
    for Rt_col in cols_list:
        Rt_time_list.append(ws.cell(Rt_x,Rt_col))
        if ws.cell(Rt_x,Rt_col).value != None:
            Rt_time_value.append(ws.cell(Rt_x,Rt_col).value)
        CAS_list.append(ws.cell(Rt_x,Rt_col+2))
        if ws.cell(Rt_x,Rt_col+2).value != None:
            CAS_val.append(ws.cell(Rt_x,Rt_col+2).value)
            

wb = openpyxl.load_workbook(file, data_only=True)
ws = wb.worksheets[0]
Rt_cols_list=[]
for Rt_time_rows in ws.iter_rows(min_row=4, max_row=4):
    for Rt_time_cell in Rt_time_rows:
        if Rt_time_cell.value == 'Rt.time':
            Rt_cols_list.append(Rt_time_cell.column)

Rt_time_list,normal_list,small_list,big_list,Rt_time_value,CAS_val,CAS_list=[],[],[],[],[],[],[]
x, Rt_x=5, 5
Rt_flg, b_flg, s_flg=True, False ,False

getRt_CAS_cols(Rt_cols_list)


while len(Rt_time_value) != 0:
    #Rttimeをリストに分ける
    CAS_num=0
    for Rt_time in Rt_time_list:
        if Rt_time.value == None:
            pass
        elif max(Rt_time_value) - 0.5 <= Rt_time.value <= max(Rt_time_value):
            big_list.append(Rt_time)
            b_flg=True
            CAS_num+=1
        elif CAS_val[CAS_num]==CAS_val[Rt_time_value.index(max(Rt_time_value))]:#範囲外でも同じ化合物なら同じとみなす
                big_list.append(Rt_time)
                CAS_num+=1
        elif min(Rt_time_value)<= Rt_time.value <= min(Rt_time_value) + 0.5:
            small_list.append(Rt_time)
            s_flg=True
            CAS_num+=1
        elif CAS_val[CAS_num]==CAS_val[Rt_time_value.index(min(Rt_time_value))]:#範囲外でも同じ化合物なら同じとみなす
                    small_list.append(Rt_time)
                    CAS_num+=1
        else :
            normal_list.append(Rt_time)
            CAS_num+=1

    s_flg=False
    b_flg=False
    #Rt.timeが小さいリスト以外を動かす
    if len(small_list) != 0:
        for b_sum in big_list:
            normal_list.append(b_sum)
        for move_col in normal_list:
            start_cel=ws.cell(x,move_col.column-1).coordinate
            end_cel= ws.cell(ws.max_row,move_col.column+4).coordinate
            ws.move_range(start_cel+':'+end_cel, rows=9)
    elif len(big_list) != 0 and len(normal_list) != 0:
        for move_col in normal_list:
            start_cel=ws.cell(x,move_col.column-1).coordinate
            end_cel= ws.cell(ws.max_row,move_col.column+4).coordinate
            ws.move_range(start_cel+':'+end_cel, rows=9)
    else:
        pass
    x += 9
    Rt_x +=9
    Rt_flg= True
    getRt_CAS_cols(Rt_cols_list)
wb.save(savepath)    
print('終了しました')

終了しました


In [4]:
"""
おまけ. 整列させたファイル上にオリジナルのデータ（シート）を追加します。
"""
#比較.xlsxのパス
copy_path = '/Users/Miyo/Desktop/ガスクロ　プログラム/4.比較/比較.xlsx'
xb2=xw.Book(copy_path)
sheet = xb2.sheets[-1]
#処理前のパス
files = sorted(glob.glob(originalfilepath + "/*.xlsx"))


for fil in files:
    xb=xw.Book(fil)
    xb.sheets[0].api.Copy(Before=sheet.api)
    xb.close()
    sheet = xb2.sheets[-1]
xb2.save()
xb2.close()
print("すべて終了しました")

すべて終了しました


In [5]:
"""
おまけ2. 　given_CAS[]に入れたCAS番が色付けされます
"""
wb4 = openpyxl.load_workbook(copy_path)
ws4 = wb4.worksheets[-1]

#CASと書かれたセルの座標を取得し、リストに作成
CAS_list = []
CAS_list.clear()
for CAS_rows in ws4.iter_rows(min_row=4, max_row=4):
    for CAS_row in CAS_rows:
        if CAS_row.value == 'CAS #':
            CAS_list.append(CAS_row.column)

#色を塗る
fill = PatternFill(patternType='solid', fgColor='FFFF66')#１６進数カラーコードHEXで好みの色に変更できます

# write in sheet
for fill_col in CAS_list:
    for fill_cells in ws4.iter_cols(min_col=fill_col, max_col=fill_col):
        for fill_cell in fill_cells:
            if fill_cell.value in given_CAS:
                ws4[fill_cell.coordinate].fill = fill

wb4.save(copy_path)
wb4.close()

print('終了しました')


終了しました
